In [1]:
import sys.process._

In [2]:
"pwd" !

/data/home/alexander.kuznetsov


0

In [3]:
"hdfs dfs -put /share/slaba05data ../alexander.kuznetsov" !

put: `slaba05data/lab05_submission_example.csv': File exists
put: `slaba05data/lab05_train.csv': File exists
put: `slaba05data/lab05_test.csv': File exists


1

In [4]:
"hdfs dfs -ls" !

Found 8 items
drwx------   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-25 21:00 .Trash
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-28 23:13 .sparkStaging
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-16 21:20 lab03.csv
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-25 11:20 lab04_KAV.pkl
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-25 12:29 lab04_KAV_lr.pkl
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-28 22:56 lab05.csv
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-28 22:04 slaba05data
drwxr-xr-x   - alexander.kuznetsov alexander.kuznetsov          0 2021-03-25 12:39 streaming


0

In [5]:
"hdfs dfs -ls ./slaba05data" !

Found 3 items
-rw-r--r--   3 alexander.kuznetsov alexander.kuznetsov     399601 2021-03-28 22:04 slaba05data/lab05_submission_example.csv
-rw-r--r--   3 alexander.kuznetsov alexander.kuznetsov   24271415 2021-03-28 22:04 slaba05data/lab05_test.csv
-rw-r--r--   3 alexander.kuznetsov alexander.kuznetsov  176160768 2021-03-28 22:04 slaba05data/lab05_train.csv


0

In [6]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@7dbcf672

In [7]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.functions.{col}

In [8]:
val spark = SparkSession.builder.appName("Spark ML").getOrCreate()

spark = org.apache.spark.sql.SparkSession@7dbcf672


org.apache.spark.sql.SparkSession@7dbcf672

In [63]:
import org.apache.spark.sql.Column
import org.apache.spark.sql.Row
import org.apache.spark.sql.Dataset

In [151]:
var trainDf = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("slaba05data/lab05_train.csv")
    //.drop("_c0", "ID")

trainDf.show(1, 0, true)

-RECORD 0--------------------------------------------
 _c0                         | 333149                
 ID                          | 479990                
 CR_PROD_CNT_IL              | 0                     
 AMOUNT_RUB_CLO_PRC          | 0.0                   
 PRC_ACCEPTS_A_EMAIL_LINK    | 0.0                   
 APP_REGISTR_RGN_CODE        | null                  
 PRC_ACCEPTS_A_POS           | 0.0                   
 PRC_ACCEPTS_A_TK            | 0.0                   
 TURNOVER_DYNAMIC_IL_1M      | 0.0                   
 CNT_TRAN_AUT_TENDENCY1M     | null                  
 SUM_TRAN_AUT_TENDENCY1M     | null                  
 AMOUNT_RUB_SUP_PRC          | 0.0                   
 PRC_ACCEPTS_A_AMOBILE       | 0.0                   
 SUM_TRAN_AUT_TENDENCY3M     | null                  
 CLNT_TRUST_RELATION         | null                  
 PRC_ACCEPTS_TK              | 0.0                   
 PRC_ACCEPTS_A_MTP           | 0.0                   
 REST_DYNAMIC_FDEP_1M       

trainDf = [_c0: int, ID: int ... 115 more fields]


[_c0: int, ID: int ... 115 more fields]

In [152]:
trainDf.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [10]:
val categoriesCols = List(
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", 
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", 
    "PACK"
)

categoriesCols = List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)


List(CLNT_TRUST_RELATION, APP_MARITAL_STATUS, APP_KIND_OF_PROP_HABITATION, CLNT_JOB_POSITION_TYPE, CLNT_JOB_POSITION, APP_DRIVING_LICENSE, APP_EDUCATION, APP_TRAVEL_PASS, APP_CAR, APP_POSITION_TYPE, APP_EMP_TYPE, APP_COMP_TYPE, PACK)

In [153]:
val numColumns = trainDf.drop(categoriesCols:_*).drop("_c0", "ID", "TARGET").columns

numColumns = Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDENCY3M, ...


Array(CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, REST_DYNAMIC_SAVE_3M, CR_PROD_CNT_VCU, REST_AVG_CUR, CNT_TRAN_MED_TENDENCY1M, AMOUNT_RUB_NAS_PRC, TRANS_COUNT_SUP_PRC, CNT_TRAN_CLO_TENDENCY1M, SUM_TRAN_MED_TENDENCY1M, PRC_ACCEPTS_A_ATM, PRC_ACCEPTS_MTP, TRANS_COUNT_NAS_PRC, CNT_ACCEPTS_MTP, CR_PROD_CNT_TOVR, CR_PROD_CNT_PIL, SUM_TRAN_CLO_TENDENCY1M, TURNOVER_CC, TRANS_COUNT_ATM_PRC, AMOUNT_RUB_ATM_PRC, TURNOVER_PAYM, AGE, CNT_TRAN_MED_TENDENCY3M, ...

In [154]:
trainDf.count()

320764

In [56]:
val CLNT_TRUST_RELATION_replaceData: Map[Any, Any] = Map("мама" -> "MOTHER", "мать" -> "MOTHER", "Мать" -> "MOTHER",
                                                         "Сын" -> "SON", "Дочь" -> "DAUGHTER", "Отец" -> "FATHER", 
                                                         "Друг" -> "FRIEND", "Брат" -> "BROTHER", "Сестра" -> "SISTER",
                                                         "Жена" -> "RELATIVE", "Муж" -> "RELATIVE", 
                                                         "Близкий ро" -> "RELATIVE", "Дальний ро" -> "OTHER")

CLNT_TRUST_RELATION_replaceData = Map(Отец -> FATHER, Мать -> MOTHER, Сестра -> SISTER, Сын -> SON, мама -> MOTHER, Брат -> BROTHER, Друг -> FRIEND, Жена -> RELATIVE, Близкий ро -> RELATIVE, мать -> MOTHER, Дальний ро -> OTHER, Дочь -> DAUGHTER, Муж -> RELATIVE)


Map(Отец -> FATHER, Мать -> MOTHER, Сестра -> SISTER, Сын -> SON, мама -> MOTHER, Брат -> BROTHER, Друг -> FRIEND, Жена -> RELATIVE, Близкий ро -> RELATIVE, мать -> MOTHER, Дальний ро -> OTHER, Дочь -> DAUGHTER, Муж -> RELATIVE)

In [57]:
trainDf.select("CLNT_TRUST_RELATION").na.replace("CLNT_TRUST_RELATION", CLNT_TRUST_RELATION_replaceData)
.groupBy("CLNT_TRUST_RELATION").count().orderBy(desc("count")).show(50)

+-------------------+------+
|CLNT_TRUST_RELATION| count|
+-------------------+------+
|               null|257829|
|             FRIEND| 22801|
|             MOTHER| 10128|
|           RELATIVE|  7476|
|              OTHER|  6055|
|            BROTHER|  5106|
|             SISTER|  5032|
|             FATHER|  2770|
|           DAUGHTER|  1845|
|                SON|  1722|
+-------------------+------+



In [51]:
val APP_MARITAL_STATUS_replaceData: Map[Any, Any] = Map("w" -> "W", "t" -> "T", "c" -> "C",
                                                         "d" -> "D", "v" -> "V", "m" -> "M", 
                                                         "n" -> "N")

APP_MARITAL_STATUS_replaceData = Map(n -> N, t -> T, m -> M, v -> V, c -> C, w -> W, d -> D)


lastException: Throwable = null


Map(n -> N, t -> T, m -> M, v -> V, c -> C, w -> W, d -> D)

In [52]:
trainDf.select("APP_MARITAL_STATUS").na.replace("APP_MARITAL_STATUS", APP_MARITAL_STATUS_replaceData)
.groupBy("APP_MARITAL_STATUS").count().orderBy(desc("count")).show(50)

+------------------+------+
|APP_MARITAL_STATUS| count|
+------------------+------+
|              null|258989|
|                 M| 29146|
|                 V| 20916|
|                 D|  7013|
|                 T|  3078|
|                 N|   850|
|                 W|   611|
|                 C|   148|
|                  |    13|
+------------------+------+



In [17]:
trainDf.select("APP_KIND_OF_PROP_HABITATION").groupBy("APP_KIND_OF_PROP_HABITATION").count().orderBy(desc("count")).show(50)

+---------------------------+------+
|APP_KIND_OF_PROP_HABITATION| count|
+---------------------------+------+
|                       null|267042|
|                         SO| 25461|
|                         JO| 12750|
|                      OTHER| 10252|
|                       RENT|  2931|
|                    NPRIVAT|  2328|
+---------------------------+------+



In [18]:
trainDf.select("CLNT_JOB_POSITION_TYPE").groupBy("CLNT_JOB_POSITION_TYPE").count().orderBy(desc("count")).show(50)

+----------------------+------+
|CLNT_JOB_POSITION_TYPE| count|
+----------------------+------+
|                  null|280242|
|            SPECIALIST| 22741|
|               MANAGER|  8464|
|           TOP_MANAGER|  6605|
|             SELF_EMPL|  2712|
+----------------------+------+



In [58]:
trainDf.select("CLNT_JOB_POSITION")
.withColumn("CLNT_JOB_POSITION_UPPER", upper('CLNT_JOB_POSITION))
.drop("CLNT_JOB_POSITION")
.withColumn("CLNT_JOB_POSITION_UPPER_1WORD", regexp_replace('CLNT_JOB_POSITION_UPPER, " ", ""))
.drop("CLNT_JOB_POSITION_UPPER")
.withColumnRenamed("CLNT_JOB_POSITION_UPPER_1WORD", "CLNT_JOB_POSITION")
.groupBy("CLNT_JOB_POSITION").count().orderBy(desc("count")).show(50)

+--------------------+------+
|   CLNT_JOB_POSITION| count|
+--------------------+------+
|                null|130371|
|            ДИРЕКТОР| 22421|
|            МЕНЕДЖЕР| 11619|
| ГЕНЕРАЛЬНЫЙДИРЕКТОР|  6318|
|        РУКОВОДИТЕЛЬ|  4047|
|             ИНЖЕНЕР|  3655|
|        ГЕН.ДИРЕКТОР|  3551|
|     ПРЕДПРИНИМАТЕЛЬ|  3286|
|           БУХГАЛТЕР|  2641|
|       ЗАМ.ДИРЕКТОРА|  2483|
|          СПЕЦИАЛИСТ|  1991|
|            ВОДИТЕЛЬ|  1986|
|    ГЛАВНЫЙБУХГАЛТЕР|  1840|
|КОММЕРЧЕСКИЙДИРЕКТОР|  1804|
|     НАЧАЛЬНИКОТДЕЛА|  1712|
|         ПРОГРАММИСТ|  1690|
|  МЕНЕДЖЕРПОПРОДАЖАМ|  1657|
|       АДМИНИСТРАТОР|  1502|
|ЗАМЕСТИТЕЛЬДИРЕКТОРА|  1498|
|            ПРОДАВЕЦ|  1471|
|            ДИЗАЙНЕР|  1461|
|             СТУДЕНТ|  1396|
|ИНДИВИДУАЛЬНЫЙПРЕ...|  1307|
|               ЮРИСТ|  1157|
|                ВРАЧ|  1142|
|   ВЕДУЩИЙСПЕЦИАЛИСТ|  1015|
|                  ИП|   998|
|ИСПОЛНИТЕЛЬНЫЙДИР...|   975|
|           ЭКОНОМИСТ|   973|
|ПРОДАВЕЦ-КОНСУЛЬТАНТ|   938|
|  ФИНАНСО

In [21]:
trainDf.select("APP_DRIVING_LICENSE").groupBy("APP_DRIVING_LICENSE").count().orderBy(desc("count")).show(50)

+-------------------+------+
|APP_DRIVING_LICENSE| count|
+-------------------+------+
|               null|268868|
|                  N| 32938|
|                  Y| 18958|
+-------------------+------+



In [54]:
val APP_EDUCATION_replaceData: Map[Any, Any] = Map("a" -> "A", "e" -> "E", "i" -> "I",
                                                   "s" -> "S", "h" -> "H")

APP_EDUCATION_replaceData = Map(e -> E, s -> S, a -> A, i -> I, h -> H)


Map(e -> E, s -> S, a -> A, i -> I, h -> H)

In [55]:
trainDf.select("APP_EDUCATION").na.replace("APP_EDUCATION", APP_EDUCATION_replaceData)
.groupBy("APP_EDUCATION").count().orderBy(desc("count")).show(50)

+-------------+------+
|APP_EDUCATION| count|
+-------------+------+
|         null|259099|
|            H| 40176|
|           SS|  9196|
|           UH|  5671|
|            S|  3557|
|           HH|  1772|
|            I|   531|
|            A|   513|
|           US|   170|
|           HI|    42|
|            E|    23|
|           AC|     9|
|           AV|     5|
+-------------+------+



In [23]:
trainDf.select("APP_TRAVEL_PASS").groupBy("APP_TRAVEL_PASS").count().orderBy(desc("count")).show(50)

+---------------+------+
|APP_TRAVEL_PASS| count|
+---------------+------+
|           null|268868|
|              N| 47829|
|              Y|  4067|
+---------------+------+



In [15]:
trainDf.select("APP_CAR").groupBy("APP_CAR").count().orderBy(desc("count")).show(50)

+-------+------+
|APP_CAR| count|
+-------+------+
|   null|268869|
|      N| 29760|
|      Y| 22135|
+-------+------+



In [24]:
trainDf.select("APP_POSITION_TYPE").groupBy("APP_POSITION_TYPE").count().orderBy(desc("count")).show(50)

+-----------------+------+
|APP_POSITION_TYPE| count|
+-----------------+------+
|             null|265984|
|       SPECIALIST| 33154|
|          MANAGER| 10546|
|      TOP_MANAGER|  7638|
|        SELF_EMPL|  3442|
+-----------------+------+



In [25]:
trainDf.select("APP_EMP_TYPE").groupBy("APP_EMP_TYPE").count().orderBy(desc("count")).show(50)

+------------+------+
|APP_EMP_TYPE| count|
+------------+------+
|        null|259778|
|     PRIVATE| 53529|
|       STATE|  5451|
|       INTER|  1206|
|          IP|   800|
+------------+------+



In [26]:
trainDf.select("APP_COMP_TYPE").groupBy("APP_COMP_TYPE").count().orderBy(desc("count")).show(50)

+-------------+------+
|APP_COMP_TYPE| count|
+-------------+------+
|         null|259778|
|      PRIVATE| 53529|
|        STATE|  5451|
|        INTER|  1206|
|           IP|   800|
+-------------+------+



In [112]:
trainDf.select("APP_EMP_TYPE", "APP_COMP_TYPE").groupBy("APP_EMP_TYPE", "APP_COMP_TYPE").count()
.orderBy(desc("count")).show(50)

+------------+-------------+------+
|APP_EMP_TYPE|APP_COMP_TYPE| count|
+------------+-------------+------+
|        null|         null|259778|
|     PRIVATE|      PRIVATE| 53529|
|       STATE|        STATE|  5451|
|       INTER|        INTER|  1206|
|          IP|           IP|   800|
+------------+-------------+------+



In [27]:
trainDf.select("PACK").groupBy("PACK").count().orderBy(desc("count")).show(50)

+----+------+
|PACK| count|
+----+------+
| 102|105756|
| K01| 69478|
| O01| 45644|
| 105| 40777|
| 107| 25108|
| 103| 22430|
| 104|  6073|
| 301|  3760|
| 101|  1645|
| 109|    83|
| M01|     5|
| 108|     4|
|null|     1|
+----+------+



In [36]:
trainDf.drop(categoriesCols:_*).summary().show(numRows = 105, truncate = 100, vertical = true)

-RECORD 0-------------------------------------------
 summary                    | count                 
 _c0                        | 320764                
 ID                         | 320764                
 CR_PROD_CNT_IL             | 320764                
 AMOUNT_RUB_CLO_PRC         | 286214                
 PRC_ACCEPTS_A_EMAIL_LINK   | 139950                
 APP_REGISTR_RGN_CODE       | 54850                 
 PRC_ACCEPTS_A_POS          | 139950                
 PRC_ACCEPTS_A_TK           | 139950                
 TURNOVER_DYNAMIC_IL_1M     | 320763                
 CNT_TRAN_AUT_TENDENCY1M    | 69535                 
 SUM_TRAN_AUT_TENDENCY1M    | 69535                 
 AMOUNT_RUB_SUP_PRC         | 286213                
 PRC_ACCEPTS_A_AMOBILE      | 139950                
 SUM_TRAN_AUT_TENDENCY3M    | 100204                
 PRC_ACCEPTS_TK             | 139950                
 PRC_ACCEPTS_A_MTP          | 139950                
 REST_DYNAMIC_FDEP_1M       | 320763          

In [155]:
val trainDf_means: Dataset[Row] = trainDf.drop(categoriesCols:_*).summary()
.filter('summary === "mean")
.drop("ID", "TARGET", "summary", "_c0")
trainDf_means.printSchema
trainDf_means.show(numRows = 105, truncate = 100, vertical = true)

root
 |-- CR_PROD_CNT_IL: string (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: string (nullable = true)
 |-- APP_REGISTR_RGN_CODE: string (nullable = true)
 |-- PRC_ACCEPTS_A_POS: string (nullable = true)
 |-- PRC_ACCEPTS_A_TK: string (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- PRC_ACCEPTS_TK: string (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: string (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CNT_ACCEPTS_TK: string (nullable = true)
 |-- REST_DYNAMIC_SAVE_3M: string (nullable = true)
 |-- CR_PROD_CNT_VCU: string (nullable = true)
 |-- REST_AVG_CUR: string

trainDf_means = [CR_PROD_CNT_IL: string, AMOUNT_RUB_CLO_PRC: string ... 99 more fields]


[CR_PROD_CNT_IL: string, AMOUNT_RUB_CLO_PRC: string ... 99 more fields]

In [156]:
var columnsAndValues = trainDf_means.columns.flatMap { c => Array(lit(c), col(c)) }
trainDf_means.printSchema()

root
 |-- CR_PROD_CNT_IL: string (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: string (nullable = true)
 |-- APP_REGISTR_RGN_CODE: string (nullable = true)
 |-- PRC_ACCEPTS_A_POS: string (nullable = true)
 |-- PRC_ACCEPTS_A_TK: string (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: string (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: string (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: string (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- PRC_ACCEPTS_TK: string (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: string (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: string (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: string (nullable = true)
 |-- CNT_ACCEPTS_TK: string (nullable = true)
 |-- REST_DYNAMIC_SAVE_3M: string (nullable = true)
 |-- CR_PROD_CNT_VCU: string (nullable = true)
 |-- REST_AVG_CUR: string

columnsAndValues = Array(CR_PROD_CNT_IL, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, CNT_ACCEPTS_TK, REST_DYNAMIC_S...


Array(CR_PROD_CNT_IL, CR_PROD_CNT_IL, AMOUNT_RUB_CLO_PRC, AMOUNT_RUB_CLO_PRC, PRC_ACCEPTS_A_EMAIL_LINK, PRC_ACCEPTS_A_EMAIL_LINK, APP_REGISTR_RGN_CODE, APP_REGISTR_RGN_CODE, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_POS, PRC_ACCEPTS_A_TK, PRC_ACCEPTS_A_TK, TURNOVER_DYNAMIC_IL_1M, TURNOVER_DYNAMIC_IL_1M, CNT_TRAN_AUT_TENDENCY1M, CNT_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, SUM_TRAN_AUT_TENDENCY1M, AMOUNT_RUB_SUP_PRC, AMOUNT_RUB_SUP_PRC, PRC_ACCEPTS_A_AMOBILE, PRC_ACCEPTS_A_AMOBILE, SUM_TRAN_AUT_TENDENCY3M, SUM_TRAN_AUT_TENDENCY3M, PRC_ACCEPTS_TK, PRC_ACCEPTS_TK, PRC_ACCEPTS_A_MTP, PRC_ACCEPTS_A_MTP, REST_DYNAMIC_FDEP_1M, REST_DYNAMIC_FDEP_1M, CNT_TRAN_AUT_TENDENCY3M, CNT_TRAN_AUT_TENDENCY3M, CNT_ACCEPTS_TK, CNT_ACCEPTS_TK, REST_DYNAMIC_S...

In [157]:
import org.apache.spark.sql.types.FloatType

In [95]:
trainDf_means.withColumn("myMap", map(columnsAndValues:_*)).select(explode($"myMap"))
.toDF("Columns", "Values")
.select("Columns", "Values")
.withColumn("Means", col("Values").cast(FloatType))
.drop("Values")
.show(110, false)

+--------------------------+------------+
|Columns                   |Means       |
+--------------------------+------------+
|CR_PROD_CNT_IL            |0.10555112  |
|AMOUNT_RUB_CLO_PRC        |0.044185933 |
|PRC_ACCEPTS_A_EMAIL_LINK  |0.0         |
|APP_REGISTR_RGN_CODE      |50.94051    |
|PRC_ACCEPTS_A_POS         |0.0         |
|PRC_ACCEPTS_A_TK          |0.0         |
|TURNOVER_DYNAMIC_IL_1M    |0.001311987 |
|CNT_TRAN_AUT_TENDENCY1M   |0.41649714  |
|SUM_TRAN_AUT_TENDENCY1M   |0.4144117   |
|AMOUNT_RUB_SUP_PRC        |0.08529494  |
|PRC_ACCEPTS_A_AMOBILE     |0.0         |
|SUM_TRAN_AUT_TENDENCY3M   |0.68878174  |
|PRC_ACCEPTS_TK            |0.0         |
|PRC_ACCEPTS_A_MTP         |0.0         |
|REST_DYNAMIC_FDEP_1M      |7.2005606E-4|
|CNT_TRAN_AUT_TENDENCY3M   |0.6914648   |
|CNT_ACCEPTS_TK            |0.0         |
|REST_DYNAMIC_SAVE_3M      |0.063789226 |
|CR_PROD_CNT_VCU           |0.031481188 |
|REST_AVG_CUR              |66162.3     |
|CNT_TRAN_MED_TENDENCY1M   |0.4441

In [158]:
val trainDf_means_map: Map[String, Double] = trainDf_means
.withColumn("myMap", map(columnsAndValues:_*)).select(explode($"myMap"))
.toDF("Columns","Values").select("Columns", "Values")
.withColumn("Means", col("Values").cast(FloatType))
.drop("Values").as[(String, Double)].collect.toMap

trainDf_means_map = Map(SUM_TRAN_CLO_TENDENCY1M -> 0.4729333221912384, LDEAL_USED_AMT_AVG_YWZ -> 0.9002152681350708, CNT_TRAN_AUT_TENDENCY3M -> 0.6914647817611694, AMOUNT_RUB_SUP_PRC -> 0.0852949395775795, REST_DYNAMIC_PAYM_3M -> 0.0768471509218216, LDEAL_ACT_DAYS_PCT_TR3 -> 0.025760464370250702, PRC_ACCEPTS_A_MTP -> 0.0, AMOUNT_RUB_CLO_PRC -> 0.044185932725667953, TURNOVER_DYNAMIC_PAYM_3M -> 0.07212407141923904, AMOUNT_RUB_NAS_PRC -> 0.02401316910982132, CR_PROD_CNT_CCFP -> 0.00490081449970603, LDEAL_TENOR_MAX -> 22.367984771728516, CNT_TRAN_SUP_TENDENCY3M -> 0.642910897731781, SUM_TRAN_AUT_TENDENCY3M -> 0.68878173828125, CNT_TRAN_MED_TENDENCY3M -> 0.701127290725708, REST_AVG_PAYM -> 7192.126953125, REST_DYNAMIC_CC_3M -> 0.0074144089594483376, DEAL_YWZ_IR_MAX -> 39....


Map(SUM_TRAN_CLO_TENDENCY1M -> 0.4729333221912384, LDEAL_USED_AMT_AVG_YWZ -> 0.9002152681350708, CNT_TRAN_AUT_TENDENCY3M -> 0.6914647817611694, AMOUNT_RUB_SUP_PRC -> 0.0852949395775795, REST_DYNAMIC_PAYM_3M -> 0.0768471509218216, LDEAL_ACT_DAYS_PCT_TR3 -> 0.025760464370250702, PRC_ACCEPTS_A_MTP -> 0.0, AMOUNT_RUB_CLO_PRC -> 0.044185932725667953, TURNOVER_DYNAMIC_PAYM_3M -> 0.07212407141923904, AMOUNT_RUB_NAS_PRC -> 0.02401316910982132, CR_PROD_CNT_CCFP -> 0.00490081449970603, LDEAL_TENOR_MAX -> 22.367984771728516, CNT_TRAN_SUP_TENDENCY3M -> 0.642910897731781, SUM_TRAN_AUT_TENDENCY3M -> 0.68878173828125, CNT_TRAN_MED_TENDENCY3M -> 0.701127290725708, REST_AVG_PAYM -> 7192.126953125, REST_DYNAMIC_CC_3M -> 0.0074144089594483376, DEAL_YWZ_IR_MAX -> 39....

In [159]:
trainDf = trainDf
.na.fill(trainDf_means_map)
.withColumn("CLNT_JOB_POSITION_UPPER", upper('CLNT_JOB_POSITION))
.drop("CLNT_JOB_POSITION")
.withColumn("CLNT_JOB_POSITION_UPPER_1WORD", regexp_replace('CLNT_JOB_POSITION_UPPER, " ", ""))
.drop("CLNT_JOB_POSITION_UPPER")
.withColumnRenamed("CLNT_JOB_POSITION_UPPER_1WORD", "CLNT_JOB_POSITION")
.na.replace("CLNT_TRUST_RELATION", CLNT_TRUST_RELATION_replaceData)
.na.replace("APP_MARITAL_STATUS", APP_MARITAL_STATUS_replaceData)
.na.replace("APP_EDUCATION", APP_EDUCATION_replaceData)
.drop("APP_COMP_TYPE")

trainDf = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [160]:
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.FeatureHasher

In [161]:
val CLNT_JOB_POSITION_hasher = new FeatureHasher()
  .setInputCols("CLNT_JOB_POSITION")
  .setOutputCol("CLNT_JOB_POSITION_Vec")
  .setNumFeatures(100)

val CLNT_TRUST_RELATION_indexer = new StringIndexer()
  .setInputCol("CLNT_TRUST_RELATION")
  .setOutputCol("CLNT_TRUST_RELATION_Index")
  .setHandleInvalid("keep")

// val CLNT_TRUST_RELATION_indexed = CLNT_TRUST_RELATION_indexer.fit(trainDf)

val APP_MARITAL_STATUS_indexer = new StringIndexer()
  .setInputCol("APP_MARITAL_STATUS")
  .setOutputCol("APP_MARITAL_STATUS_Index")
  .setHandleInvalid("keep")

// val APP_MARITAL_STATUS_indexed = APP_MARITAL_STATUS_indexer.fit(trainDf)

val APP_KIND_OF_PROP_HABITATION_indexer = new StringIndexer()
  .setInputCol("APP_KIND_OF_PROP_HABITATION")
  .setOutputCol("APP_KIND_OF_PROP_HABITATION_Index")
  .setHandleInvalid("keep")

// val APP_KIND_OF_PROP_HABITATION_indexed = APP_KIND_OF_PROP_HABITATION_indexer.fit(trainDf)

val CLNT_JOB_POSITION_TYPE_indexer = new StringIndexer()
  .setInputCol("CLNT_JOB_POSITION_TYPE")
  .setOutputCol("CLNT_JOB_POSITION_TYPE_Index")
  .setHandleInvalid("keep")

// val CLNT_JOB_POSITION_TYPE_indexed = CLNT_JOB_POSITION_TYPE_indexer.fit(trainDf)

val APP_DRIVING_LICENSE_indexer = new StringIndexer()
  .setInputCol("APP_DRIVING_LICENSE")
  .setOutputCol("APP_DRIVING_LICENSE_Index")
  .setHandleInvalid("keep")

// val APP_DRIVING_LICENSE_indexed = APP_DRIVING_LICENSE_indexer.fit(trainDf)

val APP_EDUCATION_indexer = new StringIndexer()
  .setInputCol("APP_EDUCATION")
  .setOutputCol("APP_EDUCATION_Index")
  .setHandleInvalid("keep")

// val APP_EDUCATION_indexed = APP_EDUCATION_indexer.fit(trainDf)

val APP_TRAVEL_PASS_indexer = new StringIndexer()
  .setInputCol("APP_TRAVEL_PASS")
  .setOutputCol("APP_TRAVEL_PASS_Index")
  .setHandleInvalid("keep")

// val APP_TRAVEL_PASS_indexed = APP_TRAVEL_PASS_indexer.fit(trainDf)

val APP_CAR_indexer = new StringIndexer()
  .setInputCol("APP_CAR")
  .setOutputCol("APP_CAR_Index")
  .setHandleInvalid("keep")

// val APP_CAR_indexed = APP_CAR_indexer.fit(trainDf)

val APP_POSITION_TYPE_indexer = new StringIndexer()
  .setInputCol("APP_POSITION_TYPE")
  .setOutputCol("APP_POSITION_TYPE_Index")
  .setHandleInvalid("keep")

// val APP_POSITION_TYPE_indexed = APP_POSITION_TYPE_indexer.fit(trainDf)

val APP_EMP_TYPE_indexer = new StringIndexer()
  .setInputCol("APP_EMP_TYPE")
  .setOutputCol("APP_EMP_TYPE_Index")
  .setHandleInvalid("keep")

// val APP_EMP_TYPE_indexed = APP_EMP_TYPE_indexer.fit(trainDf)

val PACK_indexer = new StringIndexer()
  .setInputCol("PACK")
  .setOutputCol("PACK_Index")
  .setHandleInvalid("keep")

// val PACK_indexed = PACK_indexer.fit(trainDf)

CLNT_JOB_POSITION_hasher = featureHasher_a701f05f9261
CLNT_TRUST_RELATION_indexer = strIdx_a319fb293d13
APP_MARITAL_STATUS_indexer = strIdx_3aac17aea145
APP_KIND_OF_PROP_HABITATION_indexer = strIdx_f82208c62faa
CLNT_JOB_POSITION_TYPE_indexer = strIdx_27f1549108f1
APP_DRIVING_LICENSE_indexer = strIdx_8c187e3d1c0e
APP_EDUCATION_indexer = strIdx_4dad861f2bfb
APP_TRAVEL_PASS_indexer = strIdx_d22a6a2538bc


APP_CAR_indexer: org.apache.spark.ml.feature.Stri...


strIdx_d22a6a2538bc

In [ ]:
    "CLNT_TRUST_RELATION", 
    "APP_MARITAL_STATUS",
    "APP_KIND_OF_PROP_HABITATION", 
    "CLNT_JOB_POSITION_TYPE",
    "CLNT_JOB_POSITION", // -- Hash
    "APP_DRIVING_LICENSE", 
    "APP_EDUCATION",
    "APP_TRAVEL_PASS", 
    "APP_CAR", 
    "APP_POSITION_TYPE", 
    "APP_EMP_TYPE",
    "APP_COMP_TYPE", // -- coincide w/ APP_EMP_TYPE
    "PACK"

In [162]:
val OHencoder = new OneHotEncoderEstimator()
  .setInputCols(Array("CLNT_TRUST_RELATION_Index", 
                      "APP_MARITAL_STATUS_Index", 
                      "APP_KIND_OF_PROP_HABITATION_Index", 
                      "CLNT_JOB_POSITION_TYPE_Index", 
                      "APP_DRIVING_LICENSE_Index", 
                      "APP_EDUCATION_Index", 
                      "APP_TRAVEL_PASS_Index", 
                      "APP_CAR_Index", 
                      "APP_POSITION_TYPE_Index", 
                      "APP_EMP_TYPE_Index", 
                      "PACK_Index"))
  .setOutputCols(Array("CLNT_TRUST_RELATION_Vec", 
                      "APP_MARITAL_STATUS__Vec", 
                      "APP_KIND_OF_PROP_HABITATION_Vec", 
                      "CLNT_JOB_POSITION_TYPE_Vec", 
                      "APP_DRIVING_LICENSE_Vec", 
                      "APP_EDUCATION_Vec", 
                      "APP_TRAVEL_PASS_Vec", 
                      "APP_CAR_Vec", 
                      "APP_POSITION_TYPE_Vec", 
                      "APP_EMP_TYPE_Vec", 
                      "PACK_Vec"))

OHencoder = oneHotEncoder_b3f219d4e59d


oneHotEncoder_b3f219d4e59d

In [163]:
import org.apache.spark.ml.feature.{VectorAssembler}

In [164]:
val num_vectorAssembler = new VectorAssembler()
    .setInputCols(numColumns)
    .setOutputCol("Numfeatures")

val cat_vectorAssembler = new VectorAssembler()
    .setInputCols(Array("CLNT_JOB_POSITION_Vec",
                      "CLNT_TRUST_RELATION_Vec", 
                      "APP_MARITAL_STATUS__Vec", 
                      "APP_KIND_OF_PROP_HABITATION_Vec", 
                      "CLNT_JOB_POSITION_TYPE_Vec", 
                      "APP_DRIVING_LICENSE_Vec", 
                      "APP_EDUCATION_Vec", 
                      "APP_TRAVEL_PASS_Vec", 
                      "APP_CAR_Vec", 
                      "APP_POSITION_TYPE_Vec", 
                      "APP_EMP_TYPE_Vec", 
                      "PACK_Vec"))
    .setOutputCol("Catfeatures")

val vectorAssembler = new VectorAssembler()
    .setInputCols(Array("Numfeatures", "Catfeatures"))
    .setOutputCol("features")

num_vectorAssembler = vecAssembler_dfbfd6945ada
cat_vectorAssembler = vecAssembler_10e6f4ccf7e0
vectorAssembler = vecAssembler_5baa2c414565


vecAssembler_5baa2c414565

In [165]:
import org.apache.spark.ml.{Pipeline, PipelineModel}

In [166]:
val pipeline = new Pipeline()
  .setStages(Array(CLNT_JOB_POSITION_hasher,
                   CLNT_TRUST_RELATION_indexer, 
                   APP_MARITAL_STATUS_indexer,
                   APP_KIND_OF_PROP_HABITATION_indexer, 
                   CLNT_JOB_POSITION_TYPE_indexer, 
                   APP_DRIVING_LICENSE_indexer, 
                   APP_EDUCATION_indexer, 
                   APP_TRAVEL_PASS_indexer, 
                   APP_CAR_indexer, 
                   APP_POSITION_TYPE_indexer, 
                   APP_EMP_TYPE_indexer, 
                   PACK_indexer,
                   OHencoder, 
                   num_vectorAssembler, 
                   cat_vectorAssembler, 
                   vectorAssembler))

pipeline = pipeline_29e5b3da5994


pipeline_29e5b3da5994

In [167]:
val model = pipeline.fit(trainDf)

model = pipeline_29e5b3da5994


pipeline_29e5b3da5994

In [175]:
val trainDfCleaned = model.transform(trainDf).filter(col("TARGET").isNotNull).cache
trainDfCleaned.show(numRows=5, vertical=true, truncate=10)

-RECORD 0---------------------------------------
 _c0                               | 333149     
 ID                                | 479990     
 CR_PROD_CNT_IL                    | 0          
 AMOUNT_RUB_CLO_PRC                | 0.0        
 PRC_ACCEPTS_A_EMAIL_LINK          | 0.0        
 APP_REGISTR_RGN_CODE              | 50.9405... 
 PRC_ACCEPTS_A_POS                 | 0.0        
 PRC_ACCEPTS_A_TK                  | 0.0        
 TURNOVER_DYNAMIC_IL_1M            | 0.0        
 CNT_TRAN_AUT_TENDENCY1M           | 0.41649... 
 SUM_TRAN_AUT_TENDENCY1M           | 0.41441... 
 AMOUNT_RUB_SUP_PRC                | 0.0        
 PRC_ACCEPTS_A_AMOBILE             | 0.0        
 SUM_TRAN_AUT_TENDENCY3M           | 0.68878... 
 CLNT_TRUST_RELATION               | null       
 PRC_ACCEPTS_TK                    | 0.0        
 PRC_ACCEPTS_A_MTP                 | 0.0        
 REST_DYNAMIC_FDEP_1M              | 0.0        
 CNT_TRAN_AUT_TENDENCY3M           | 0.69146... 
 CNT_ACCEPTS_TK     

trainDfCleaned = [_c0: int, ID: int ... 140 more fields]


[_c0: int, ID: int ... 140 more fields]

In [176]:
trainDfCleaned.count()

320763

In [148]:
// trainDf = trainDf.drop(categoriesCols:_*).na.fill(0)

Name: Syntax Error.
Message: 
StackTrace: 

In [177]:
val Array(trainDfTrain, trainDfTest) = trainDfCleaned.select("TARGET", "features").randomSplit(Array(0.8, 0.2))

trainDfTrain = [TARGET: int, features: vector]
trainDfTest = [TARGET: int, features: vector]


[TARGET: int, features: vector]

In [190]:
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
import org.apache.spark.ml.feature.{VectorAssembler}

In [186]:
// Train RandomForest
val randomForestModel = new RandomForestClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(42)
    .setNumTrees(50)
    .fit(trainDfTrain)

randomForestModel = RandomForestClassificationModel (uid=rfc_2d5c121a0508) with 50 trees


RandomForestClassificationModel (uid=rfc_2d5c121a0508) with 50 trees

In [187]:
val predictions = randomForestModel.transform(
    trainDfTest
).cache

predictions
    .select(
        "TARGET",
        "features",
        "rawPrediction",
        "probability",
        "prediction"
    )
    .show(5)

+------+--------------------+--------------------+--------------------+----------+
|TARGET|            features|       rawPrediction|         probability|prediction|
+------+--------------------+--------------------+--------------------+----------+
|     0|(265,[0,1,3,6,7,8...|[44.9852957013640...|[0.89970591402728...|       0.0|
|     0|(265,[0,1,3,6,7,8...|[45.9808546034820...|[0.91961709206964...|       0.0|
|     0|(265,[0,1,3,6,7,8...|[45.2220093974222...|[0.90444018794844...|       0.0|
|     0|(265,[0,1,3,6,7,8...|[45.3585825205707...|[0.90717165041141...|       0.0|
|     0|(265,[0,1,3,6,7,8...|[45.4148124696025...|[0.90829624939205...|       0.0|
+------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



predictions = [TARGET: int, features: vector ... 3 more fields]


[TARGET: int, features: vector ... 3 more fields]

In [191]:
// Train GBTClassifier
val GBTCModel = new GBTClassifier()
    .setLabelCol("TARGET")
    .setFeaturesCol("features")
    .setSeed(42)
    .setMaxIter(50)
    .fit(trainDfTrain)

GBTCModel = GBTClassificationModel (uid=gbtc_ae61efa2b97b) with 50 trees


GBTClassificationModel (uid=gbtc_ae61efa2b97b) with 50 trees

In [193]:
val predictions = GBTCModel.transform(
    trainDfTest
).cache

predictions = [TARGET: int, features: vector ... 3 more fields]


[TARGET: int, features: vector ... 3 more fields]

In [188]:
predictions
    .select(
        "probability"
    )
    .show(1, 0, true)

-RECORD 0----------------------------------------------
 probability | [0.899705914027281,0.10029408597271904] 
only showing top 1 row



In [181]:
predictions.select("probability").printSchema()

root
 |-- probability: vector (nullable = true)



In [182]:
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.mllib.linalg.{Vector => OldVector}

val vectorToArrayUdf = udf { vec: Any =>
    vec match {
      case v: Vector => v.toArray
      case v: OldVector => v.toArray
      case v => throw new IllegalArgumentException(
        "function vector_to_array requires a non-null input argument and input type must be " +
        "`org.apache.spark.ml.linalg.Vector` or `org.apache.spark.mllib.linalg.Vector`, " +
        s"but got ${ if (v == null) "null" else v.getClass.getName }.")
    }
  }.asNonNullable()

vectorToArrayUdf = UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)


import org.apache.spark.mllib.linalg.{Vector=>OldVector}


UserDefinedFunction(<function1>,ArrayType(DoubleType,false),None)

In [183]:
predictions
    .select(vectorToArrayUdf(col("probability")).getItem(1).alias("prediction_probability"))
    .show(5)

+----------------------+
|prediction_probability|
+----------------------+
|   0.09551577790293662|
|    0.0761206279835468|
|   0.09597905844312773|
|   0.08906888218239482|
|   0.09592983966605001|
+----------------------+
only showing top 5 rows



In [184]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [194]:
val evaluator = new BinaryClassificationEvaluator()
    .setLabelCol("TARGET")
    .setRawPredictionCol("prediction")
    .setMetricName("areaUnderROC")

val accuracy = evaluator
    .evaluate(
        predictions.withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    )

evaluator = binEval_847f19090fa3
accuracy = 0.8332907597952605


0.8332907597952605

In [211]:
var testDfData = spark
    .read
    .option("header", true)
    .option("inferSchema", true)
    .csv("slaba05data/lab05_test.csv")

testDfData.show(1, 0, vertical=true)

-RECORD 0-----------------------------------------
 _c0                         | 372289             
 ID                          | 519130             
 CR_PROD_CNT_IL              | 0                  
 AMOUNT_RUB_CLO_PRC          | 0.0                
 PRC_ACCEPTS_A_EMAIL_LINK    | null               
 APP_REGISTR_RGN_CODE        | null               
 PRC_ACCEPTS_A_POS           | null               
 PRC_ACCEPTS_A_TK            | null               
 TURNOVER_DYNAMIC_IL_1M      | 0.0                
 CNT_TRAN_AUT_TENDENCY1M     | null               
 SUM_TRAN_AUT_TENDENCY1M     | null               
 AMOUNT_RUB_SUP_PRC          | 0.0                
 PRC_ACCEPTS_A_AMOBILE       | null               
 SUM_TRAN_AUT_TENDENCY3M     | null               
 CLNT_TRUST_RELATION         | null               
 PRC_ACCEPTS_TK              | null               
 PRC_ACCEPTS_A_MTP           | null               
 REST_DYNAMIC_FDEP_1M        | 0.0                
 CNT_TRAN_AUT_TENDENCY3M     | 

testDfData = [_c0: int, ID: int ... 114 more fields]


[_c0: int, ID: int ... 114 more fields]

In [212]:
testDfData.printSchema

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- CR_PROD_CNT_IL: integer (nullable = true)
 |-- AMOUNT_RUB_CLO_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_EMAIL_LINK: double (nullable = true)
 |-- APP_REGISTR_RGN_CODE: double (nullable = true)
 |-- PRC_ACCEPTS_A_POS: double (nullable = true)
 |-- PRC_ACCEPTS_A_TK: double (nullable = true)
 |-- TURNOVER_DYNAMIC_IL_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY1M: double (nullable = true)
 |-- AMOUNT_RUB_SUP_PRC: double (nullable = true)
 |-- PRC_ACCEPTS_A_AMOBILE: double (nullable = true)
 |-- SUM_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CLNT_TRUST_RELATION: string (nullable = true)
 |-- PRC_ACCEPTS_TK: double (nullable = true)
 |-- PRC_ACCEPTS_A_MTP: double (nullable = true)
 |-- REST_DYNAMIC_FDEP_1M: double (nullable = true)
 |-- CNT_TRAN_AUT_TENDENCY3M: double (nullable = true)
 |-- CNT_ACCEPTS_TK: double (nullable = true)
 

In [213]:
testDfData = testDfData
    .na.fill(trainDf_means_map)
    .withColumn("CLNT_JOB_POSITION_UPPER", upper('CLNT_JOB_POSITION))
    .drop("CLNT_JOB_POSITION")
    .withColumn("CLNT_JOB_POSITION_UPPER_1WORD", regexp_replace('CLNT_JOB_POSITION_UPPER, " ", ""))
    .drop("CLNT_JOB_POSITION_UPPER")
    .withColumnRenamed("CLNT_JOB_POSITION_UPPER_1WORD", "CLNT_JOB_POSITION")
    .na.replace("CLNT_TRUST_RELATION", CLNT_TRUST_RELATION_replaceData)
    .na.replace("APP_MARITAL_STATUS", APP_MARITAL_STATUS_replaceData)
    .na.replace("APP_EDUCATION", APP_EDUCATION_replaceData)
    .drop("APP_COMP_TYPE")

testDfData = [_c0: int, ID: int ... 113 more fields]


[_c0: int, ID: int ... 113 more fields]

In [214]:
val testDfCleaned = model.transform(testDfData).cache

testDfCleaned = [_c0: int, ID: int ... 139 more fields]


[_c0: int, ID: int ... 139 more fields]

In [215]:
val realPredictions = randomForestModel.transform(testDfCleaned).cache

realPredictions = [_c0: int, ID: int ... 142 more fields]


[_c0: int, ID: int ... 142 more fields]

In [216]:
val realPredictions = GBTCModel.transform(testDfCleaned).cache

realPredictions = [_c0: int, ID: int ... 142 more fields]


[_c0: int, ID: int ... 142 more fields]

In [217]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .show(5)

+------+--------------------+
|    ID|          prediction|
+------+--------------------+
|519130|  0.1916535205040647|
|234045|0.028549333891959572|
|401256|0.025250971858566507|
|551070|0.056142034782829864|
|367285|0.030546019855911544|
+------+--------------------+
only showing top 5 rows



In [221]:
realPredictions
    .withColumn("prediction", vectorToArrayUdf(col("probability")).getItem(1))
    .select("ID", "prediction")
    .withColumnRenamed("prediction", "TARGET")
    .withColumnRenamed("ID", "id")
    .withColumnRenamed("TARGET", "target")
    .repartition(1)
    .write
    .mode("overwrite")
    .format("csv")
    .option("delimiter", "\t")
    .option("header", "true")
    .save("lab05.csv")

In [222]:
"hdfs dfs -ls lab05.csv" !

Found 2 items
-rw-r--r--   3 alexander.kuznetsov alexander.kuznetsov          0 2021-03-29 18:27 lab05.csv/_SUCCESS
-rw-r--r--   3 alexander.kuznetsov alexander.kuznetsov    1202255 2021-03-29 18:27 lab05.csv/part-00000-c08b584d-7cef-4b76-9339-2ab7c6fb94c4-c000.csv


0

In [225]:
"hdfs dfs -get lab05.csv/part-00000-c08b584d-7cef-4b76-9339-2ab7c6fb94c4-c000.csv /data/home/alexander.kuznetsov/lab05.csv" !

0

In [226]:
"ls" !

check_file.ipynb
custom_regex_transformer_skeleton.ipynb
lab01.ipynb
lab01.json
lab02.ipynb
lab02.json
lab03.csv
lab03.ipynb
lab04.ipynb
lab05.csv
lab05.ipynb


0

In [227]:
spark.stop()